# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [16]:
from aimakerspace.text_utils import CharacterTextSplitter, TextFileUtils
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [17]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileUtils` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [18]:
text_loader = TextFileUtils("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [19]:
print(documents[0][:202])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horowitz
cover design: Jessica Hagy
produced using: Pressbooks
Contents
THE PMARCA GUIDE TO STARTUPS
Part 1


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [20]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [21]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [22]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?

2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. Yes, according to the [OpenAI's Embeddings guide](https://platform.openai.com/docs/guides/embeddings),
 > By default, the length of the embedding vector is 1536 for text-embedding-3-small or 3072 for text-embedding-3-large. To reduce the embedding's dimensions without losing its concept-representing properties, pass in the dimensions parameter.
2. According to OpenAI's Embeddings guide,
> OpenAI new embedding models were trained with a technique that allows developers to trade-off performance and cost of using embeddings. Specifically, developers can shorten embeddings (i.e. remove some numbers from the end of the sequence) without the embedding losing its concept-representing properties by passing in the dimensions API parameter.
The techinique mentioned by the OpenAI Embeddings guide is know as the [**Matryoshka Representation Learning (MRL)**](https://arxiv.org/abs/2205.13147) which encodes information at different granularities and allows a single embedding to adapt to the computational constraints of downstream tasks. The MRL datasets and pretrained models are Open Sourced and available at https://github.com/RAIVNLab/MRL.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [23]:
vector_db = VectorDatabase()
vector_db = await vector_db.abuild_from_list(split_documents)

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
The `async` keyword is used to define **asynchronous** functions in Python. When called, an async function returns a coroutine object, which can be **executed without blocking the caller**. Instead of running in a separate thread or process, async functions run within the same thread but cooperate with an event loop, allowing other tasks to run while waiting for slow operations (like network requests or file I/O) to complete. This makes asynchronous programming ideal for I/O-bound tasks, where waiting does not consume CPU unnecessarily.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

#### using the default Cosine Similarity as the verctor distance measurement

In [24]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

#### using a different distance measurment

In [25]:
from aimakerspace.vectordatabase import DistanceMeasure
vector_db.search_by_text(
    "What is the Michael Eisner Memorial Weak Executive Problem?", 
    k=3, 
    distance_measure=DistanceMeasure.EUCLIDEAN)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
The documentation makes a couple of recommendations to get good results "consistently".
 * Prefer to use the new Responses API, especially if using Reasoning Models or building text generation apps.
 * Use **Message roles and instruction following**
 > You can provide instructions to the model with differing levels of authority using the instructions API parameter along with message roles.
 > The instructions parameter gives the model high-level instructions on how it should behave while generating a response, including tone, goals, and examples of correct responses. Any instructions provided this way will take priority over a prompt in the input parameter.
 * Use of [Reusable prompts](https://platform.openai.com/docs/guides/text?prompt-templates-examples=simple#reusable-prompts)

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [26]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [27]:
print(response)

Hello! The best way to write a loop in Python depends on what you want to achieve. Generally, Python offers a couple of common loop constructs:

1. **`for` loop:** Best when you want to iterate over a sequence (like a list, tuple, string, or range of numbers).

```python
for item in iterable:
    # do something with item
```

Example:

```python
for i in range(5):
    print(i)
```

2. **`while` loop:** Best when you want to repeat something as long as a condition is true.

```python
while condition:
    # do something
```

Example:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

For readability and Pythonic style, the `for` loop is often preferred when possible, especially when iterating over a fixed sequence or range. Using meaningful variable names and keeping the loop body clear also makes your loop "best" in practice.

If you have a specific task or scenario in mind, I’d be happy to help craft the most suitable loop for it! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [28]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [39]:
import pprint
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 search_with_metadata: bool = False, response_style: str = "detailed", 
                 include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.search_with_metadata = search_with_metadata
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        if self.search_with_metadata:
            context_list = self.vector_db_retriever.search_with_metadata(user_query, k=k)
        else:
            context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, result in enumerate(context_list, 1):
            if self.search_with_metadata:
                # When using metadata, result is a dictionary
                context = result.get('text', result.get('key', ''))
                score = result.get('similarity_score', 0.0)
            else:
                # When not using metadata, result is a tuple (context, score)
                context, score = result
            
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        # pprint.pprint({"Formated RAG System Prompt": formatted_system_prompt})

        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)
        # pprint.pprint({"Formated RAG User Prompt": formatted_user_prompt})

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [30]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a common issue where a CEO or startup founder, who previously excelled in a particular function (e.g., product management, sales, marketing), ends up hiring a weak executive to run that same function. This happens because the CEO has a hard time letting go of the function that originally brought them success, and as a result, they hire someone weak in that role to maintain their own dominance or control over it. The name comes from Michael Eisner, the former CEO of Disney, who had been a brilliant TV network executive. When he acquired ABC at Disney, ABC quickly fell to fourth place. Eisner claimed that if he had an extra two days a week, he could have turned ABC around himself. However, he did not have that time, illustrating the problem that the CEO cannot simply continue to personally manage what they used to do best and that hiring weak executives in that role is detrimental. (Source 1)

Context Count: 3
Simil

#### ❓ Question #4:
What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
Use of **Message roles and instruction following**, choose a Reasoning model and switch to teh new Responses API.

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files ✅
- Implement a new distance metric ✅
- Add metadata support to the vector database ✅
- Use a different embedding model
- Add the capability to ingest a YouTube link ✅

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

#### Ingesting a PDF into the Vector DB

1. Extract PDF content and convert it to Markdown format

In [31]:
from docling.document_converter import DocumentConverter
from aimakerspace.text_utils import TextFileUtils

# PDF used as sample: "Principled Instructions Are All You Need for Questioning LLaMA-1/2, GPT-3.5/4" paper.
pdf_source = "https://arxiv.org/pdf/2312.16171"  # document per local path or URL
converter = DocumentConverter()
doc = converter.convert(pdf_source).document

# Save PDF Content as Markdown
data_path = "data/"
text_file_utils = TextFileUtils(data_path)
print(f"Saving Markdown content for {doc.name} [source: {pdf_source}]\n")
text_file_utils.save_file(doc.export_to_markdown(), doc.name + ".md.tmp")

2025-09-17 20:12:35,597 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-17 20:12:35,638 - INFO - Going to convert document batch...
2025-09-17 20:12:35,638 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-17 20:12:35,643 - INFO - Loading plugin 'docling_defaults'
2025-09-17 20:12:35,644 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-17 20:12:35,648 - INFO - Loading plugin 'docling_defaults'
2025-09-17 20:12:35,650 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-17 20:12:35,809 - INFO - Accelerator device: 'mps'
2025-09-17 20:12:38,152 - INFO - Accelerator device: 'mps'
2025-09-17 20:12:39,197 - INFO - Accelerator device: 'mps'
2025-09-17 20:12:39,820 - INFO - Processing document 2312.16171v2.pdf
2025-09-17 20:12:53,902 - INFO - Finished converting document 2312.16171v2.pdf in 18.70 sec.


Saving Markdown content for 2312.16171v2 [source: https://arxiv.org/pdf/2312.16171]



2. Chunk the Markdown content

In [32]:
from docling.chunking import HybridChunker
import tiktoken
from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer

tokenizer = OpenAITokenizer(
    tokenizer=tiktoken.encoding_for_model("text-embedding-3-small"),
    max_tokens=8091,  # context window length required for OpenAI tokenizers
)

chunker = HybridChunker(tokenizer=tokenizer)
chunk_iter = chunker.chunk(dl_doc=doc)
chunks = [chunker.contextualize(chunk=chunk) for chunk in chunk_iter]
print(f"Document split into {len(chunks)} chunks")
# ser_txt = chunker.contextualize(chunk=chunks[0])
# print(f"Number of tokens in the 1st chunk: {len(ser_txt)}")


Document split into 26 chunks


3. Insert chuncks into the Vector DB

In [33]:
import asyncio
from aimakerspace.vectordatabase import VectorDatabase

vector_db = VectorDatabase()
vector_db = await vector_db.abuild_from_list(chunks)

2025-09-17 20:12:55,524 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


4. Query the Vector DB

In [34]:
vector_db.search_by_text("Which is the intended audience in the prompt?", k=3)

2025-09-17 20:12:56,190 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[("3.2 Overview\nThe overview of principles is presented in Table 1. According to their unique nature, we group them into five categories as in Table 2: (1) Prompt Structure and Clarity, e.g., integrate the intended audience in the prompt such as the audience is an expert in the field ; (2) Specificity and Information, e.g., Add to your prompt the following phrase 'Ensure that your answer is unbiased and does not rely on stereotypes. ' ; (3) User Interaction and Engagement, e.g., Allow the model to elicit precise details and requirements from you by asking you questions until he has enough information to provide the needed output 'From now on, I would like you to ask me questions to... '. (4) Content and Language Style, e.g., No need to be polite with LLM so there is no need to add phrases like 'please', 'if you don't mind', 'thank you', 'I would like to', etc., and get straight to the point ; (5) Complex Tasks and Coding Prompts, e.g., Break down complex tasks into a sequence of simpl

5. Create a RAG Pipeline using our extracted PDF contentas data source

In [35]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "List the 26 ordered prompt principles contained in the document",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

2025-09-17 20:12:56,654 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-17 20:13:22,971 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Based on the provided context, the 26 prompt principles for instructions as outlined and presented in Table 1 (Source 1) are as follows:

1. If you prefer more concise answers, no need to be polite with LLM so there is no need to add phrases like "please", "if you don't mind", "thank you", "I would like to", etc., and get straight to the point.
2. Integrate the intended audience in the prompt, e.g., the audience is an expert in the field.
3. Break down complex tasks into a sequence of simpler prompts in an interactive conversation.
4. Employ affirmative directives such as "do," while steering clear of negative language like "don't."
5. When you need clarity or a deeper understanding of a topic, idea, or any piece of information, utilize prompts such as:
   - Explain [insert specific topic] in simple terms.
   - Explain to me like I'm 11 years old.
   - Explain to me as if I'm a beginner in [field].
6. Add "I'm going to tip $xxx for a better solution!"
7. Implement example-dri

#### Ingesting a YouTube video into the Vector DB

In [36]:
# First, install required packages:
# pip install yt-dlp openai-whisper

import asyncio
import shutil
import os
from aimakerspace.youtube_utils import YouTubeProcessor
from aimakerspace.vectordatabase import VectorDatabase

# Example YouTube URL (replace with actual video)
YT_VIDEO_URL = "https://www.youtube.com/watch?v=aircAruvnKk"

async def build_youtube_db(video_url: str):
    # Create vector database and YouTube processor
    vector_db = VectorDatabase()
    processor = YouTubeProcessor()

    # Ingest the video (this will download, transcribe, and embed)
    print("Ingesting YouTube video...")
    result = await processor.ingest_youtube_video(vector_db, video_url, chunk_duration=30)
    
    print(f"✅ Ingested {result['segments_ingested']} segments")
    print(f"▶️ Video: {result['video_title']}")
    print(f"⏱️  Duration: {result['total_duration']:.1f} seconds")
    
    return vector_db
    
# Run the demo
youtube_db = await build_youtube_db(YT_VIDEO_URL)


2025-09-17 20:13:23,275 - INFO - Initializing YouTubeProcessor with Whisper model: base
2025-09-17 20:13:23,825 - INFO - ✅ ffmpeg available at: /opt/homebrew/bin/ffmpeg
2025-09-17 20:13:23,826 - INFO - ✅ ffprobe available at: /opt/homebrew/bin/ffprobe
2025-09-17 20:13:23,826 - INFO - Starting YouTube video ingestion for: https://www.youtube.com/watch?v=aircAruvnKk
2025-09-17 20:13:23,826 - INFO - Using chunk duration: 30 seconds
2025-09-17 20:13:23,826 - INFO - Creating video segments for: https://www.youtube.com/watch?v=aircAruvnKk


Ingesting YouTube video...


2025-09-17 20:13:27,404 - INFO - Downloading audio...
2025-09-17 20:13:27,405 - INFO - Starting audio download for URL: https://www.youtube.com/watch?v=aircAruvnKk
2025-09-17 20:13:27,406 - INFO - Extracting video metadata...
2025-09-17 20:13:29,749 - INFO - Video title: But what is a neural network? | Deep learning chapter 1
2025-09-17 20:13:29,751 - INFO - Audio file already exists, using existing file: data/temp_audio/But-what-is-a-neural-network-Deep-learning-chapter-1.wav
2025-09-17 20:13:29,751 - INFO - Transcribing audio...
2025-09-17 20:13:29,751 - INFO - Starting audio transcription for: data/temp_audio/But-what-is-a-neural-network-Deep-learning-chapter-1.wav
2025-09-17 20:13:29,751 - INFO - Transcribing with Whisper model: base
/Users/rsoares/dev/github/rafaeltuelho/AIE8/02_Embeddings_and_RAG/.venv/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 inst

✅ Ingested 45 segments
▶️ Video: But what is a neural network? | Deep learning chapter 1
⏱️  Duration: 1121.0 seconds


2. Query the Vector DB

In [37]:
# Search for relevant content
query = "What is a layer in a neural network?"
print(f"\n🔍 Searching for: '{query}'")

results = youtube_db.search_with_metadata(query, k=3)

for i, result in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print(f"📝 Text: {result['text'][:150]}...")
    print(f"⏰ Time: {result['start_time']:.1f}s - {result['end_time']:.1f}s")
    print(f"🎯 Relevance: {result['similarity_score']:.3f}")
    print(f"🔗 Video timestamp: {YT_VIDEO_URL}&t={result['start_time']}s")


🔍 Searching for: 'What is a layer in a neural network?'


2025-09-17 20:14:24,458 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



--- Result 1 ---
📝 Text: of the next layer. And of course, the heart of the network, as an information processing mechanism, comes down to exactly how those activations from o...
⏰ Time: 275.0s - 305.0s
🎯 Relevance: 0.627
🔗 Video timestamp: https://www.youtube.com/watch?v=aircAruvnKk&t=275.0s

--- Result 2 ---
📝 Text: you feed in. So it's actually more accurate to think of each neuron as a function, one that takes in the outputs of all the neurons in the previous la...
⏰ Time: 925.0s - 955.0s
🎯 Relevance: 0.568
🔗 Video timestamp: https://www.youtube.com/watch?v=aircAruvnKk&t=925.0s

--- Result 3 ---
📝 Text: Then organize all of the weights as a matrix, where each row of that matrix corresponds to the connections between one layer and a particular neuron i...
⏰ Time: 825.0s - 855.0s
🎯 Relevance: 0.555
🔗 Video timestamp: https://www.youtube.com/watch?v=aircAruvnKk&t=825.0s


3. Create a RAG Pipeline using our Video transcription

In [42]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=youtube_db,
    search_with_metadata=True,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "what are Layers in a neural network?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

2025-09-17 20:20:49,629 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-17 20:24:16,069 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Layers in a neural network are structured groups of neurons (or functions) where the activations (outputs) from one layer influence the activations of the next layer. Each neuron can be considered as a function that takes in the outputs from all neurons in the previous layer and produces a number between 0 and 1. Collectively, the entire network functions as a complex function that takes an input (e.g., 784 numbers representing an image) and outputs a result (e.g., 10 numbers corresponding to digit predictions) [Source 1, Source 2]. The layered structure is meant to be loosely analogous to biological neural networks, where some neurons firing cause others to fire, enabling the network to process information progressively across layers [Source 1].

Context Count: 3
Similarity Scores: ['Source 1: 0.610', 'Source 2: 0.558', 'Source 3: 0.552']
